In [155]:
import requests
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd
import parser
import re

In [235]:
site = 'https://www.hanbit.co.kr'
url = 'https://www.hanbit.co.kr/store/books/new_book_list.html'
driver = webdriver.Chrome('chromedriver.exe')
driver.get(url)
req = requests.get(url)
html = req.text
time.sleep(2)

In [236]:
lis = soup.select('.new_book_list_wrap>div>li')
len(lis)

20

In [237]:
href = li.find('a')['href']
href

'/store/books/look.php?p_code=B6694310162'

In [238]:
sub_url = site+href
sub_url

'https://www.hanbit.co.kr/store/books/look.php?p_code=B6694310162'

In [239]:
sub_req = requests.get(sub_url)
sub_html = sub_req.text
sub_soup = BeautifulSoup(sub_html,'html.parser')

In [240]:
lis = soup.select('.new_book_list_wrap>div>li')
len(lis)

20

In [241]:
li = lis[0]

In [242]:
href = li.find('a')['href']
href

'/store/books/look.php?p_code=B6694310162'

In [243]:
sub_url = site+href
sub_url

'https://www.hanbit.co.kr/store/books/look.php?p_code=B6694310162'

In [244]:
itemName = li.select_one('.book_tit').get_text()
itemName

'하루 한 장 초등과학 365'

In [245]:
writer = li.select_one('.book_writer').get_text()
writer

'치바 가즈요시 '

In [246]:
#역자
translator = sub_soup.select_one('.info_list>li:nth-child(2)').get_text().split(':')[1]
translator

' 허영은 , 유우종 (감수) '

In [277]:
translator = sub_soup.select_one('.info_list')
len(translator)==13

True

In [123]:
soup = BeautifulSoup(driver.page_source,'html.parser')


In [307]:
#출간일
date = sub_soup.select_one('.info_list').get_text().split('출간')[1][:15]
date = re.sub('[^0-9]','',date)
date = int(date)
date

20211102

In [324]:
#페이지
jpage=sub_soup.select_one('.info_list').get_text().split('쪽')[0][-5:]
jpage = re.sub('[^0-9]','',jpage)
jpage = int(jpage)
jpage

96

In [180]:
#정가
price=sub_soup.select_one('.store_payment_area>fieldset>label>p:nth-child(1)').get_text()
price = re.sub('[^0-9]','',price)
price = int(price)
price

19800

In [191]:
#할인가
sale=sub_soup.select_one('.store_payment_area>fieldset>label>p:nth-child(2)>span:nth-child(2)>strong').get_text()
sale = re.sub('[^0-9]','',sale)
sale = int(sale)
sale

17820

In [201]:
#마일리지
Mileage=sub_soup.select_one('.store_payment_area>fieldset>label>p:nth-child(3)>span:nth-child(2)').get_text().split('점')[0]
Mileage = int(Mileage)
Mileage

990

In [331]:
lines=[]

site = 'https://www.hanbit.co.kr'
url = 'https://www.hanbit.co.kr/store/books/new_book_list.html'
driver = webdriver.Chrome('chromedriver.exe')
driver.get(url)
req = requests.get(url)
html = req.text
time.sleep(1)
for page in range(1,11):
    url = 'https://www.hanbit.co.kr/store/books/new_book_list.html?page='+str(page)+'&brand=&cate1=&cate2=&searchKey=&keyWord='
    driver.get(url)
    time.sleep(1)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    lis = soup.select('.new_book_list_wrap>div>li')

    for li in lis:
        href = li.find('a')['href']
        sub_url = site+href
        sub_req = requests.get(sub_url)
        sub_html = sub_req.text
        sub_soup = BeautifulSoup(sub_html,'html.parser')
        
        writer = li.select_one('.book_writer').get_text()
        itemName = li.select_one('.book_tit').get_text()

        Mileage=sub_soup.select_one('.store_payment_area>fieldset>label>p:nth-child(3)>span:nth-child(2)').get_text().split('점')[0]
        Mileage = re.sub('[^0-9]','',Mileage)
        Mileage=int(Mileage)
        sale=sub_soup.select_one('.store_payment_area>fieldset>label>p:nth-child(2)>span:nth-child(2)>strong').get_text()
        sale = re.sub('[^0-9]','',sale)
        sale = int(sale)
        price=sub_soup.select_one('.store_payment_area>fieldset>label>p:nth-child(1)').get_text()
        price = re.sub('[^0-9]','',price)
        price = int(price)
        jpage=sub_soup.select_one('.info_list').get_text().split('쪽')[0][-5:]
        jpage = re.sub('[^0-9]','',jpage)
        jpage = int(jpage)
        date = sub_soup.select_one('.info_list').get_text().split('출간')[1][:15]
        date = re.sub('[^0-9]','',date)
        date = int(date)
        date
        if len(sub_soup.select_one('.info_list'))==13:
            translator = sub_soup.select_one('.info_list>li:nth-child(2)').get_text().split(':')[1]
            translator = re.sub('[0-9]','',translator)
            pass
        else:
            translator ='--'
            pass
        lines.append([itemName,writer,translator,date,jpage,price,sale,Mileage])

driver.close()

In [332]:
df = pd.DataFrame(lines, columns=['제목', '저자', '역자', '출간일', '페이지', '정가', '판가','마일리지'])

In [335]:
df.tail(5)

,제목,저자,역자,출간일,페이지,정가,판가,마일리지
195,"IT CookBook, C 언어 일취월장(2판)",신윤환,--,20210224,800,28000,28000,840
196,소비자행동,김학윤,--,20210223,660,35000,35000,1050
197,재미있고 빠른 어휘력 동화 – 두근두근 이 마음은 뭘까,"김세실 , 김도윤(그림)",--,20210220,104,12000,10800,600
198,재미있고 빠른 어휘력 동화 – 키득키득 재밌는 의성어 의태어,"김예실 , 김희선(그림)",--,20210220,104,12000,10800,600
199,웹 애플리케이션 보안,앤드루 호프먼,최용,20210219,372,30000,27000,1500


In [334]:
df.to_csv('한빛도서과제.csv', index=False)